In [ ]:
import operator 
import os
import numpy as np
from skimage import io, draw, transform, exposure, color

# цвета для темной и светлой областей 
BRIGHT = 240
DARK = 50

def main():
  dataset = read_images('./cars') # Считывание машин
  for im_no, image in enumerate(dataset):
    coordinates = detect_object(image[1], 0.3) # детекция объектов 
    save_result(image[0], coordinates, im_no) # отображение результата


# функция загрузки и обработки картинок 
def read_images(path):
  dataset = []
  for image_name in os.listdir(path):
    img = io.imread(path + '\\' + image_name)
    img_gray = color.rgb2gray(img) # делаем однотонное изображение 
    img_contrast = exposure.adjust_sigmoid(img_gray, cutoff=0.5, gain=25,
                                           inv=False) # сохраняем высоко-онтрастную версию изображения
                                           dataset.append((img, img_contrast)) # добавляем в наш датасет новую картинку
  return dataset

In [ ]:
def imsave(path, no, img): # функция сохранения картинки
  return io.imsave(path + str(no) + '.jpg', img)


# Создаем характерный признак Хаара для машин
# Светлая область, темная узкая, шире в три раза светлая # dark_w - ширина темной грани
# bright_w - ширина узкой светлой грани # h - высота
def get_haar_params(bright_w, dark_w, h): 
  return [[(0, 0), (h, bright_w)],[(0, bright_w), (h, dark_w + bright_w)],[(0, bright_w + dark_w), (h, 4 * bright_w + dark_w)]]


def detect_object(hc_img, w_threshold): 
  res_coords = [] 
  # Это интегральное изображение, необходимое для быстрого расчета признака Хаара
  int_img = transform.integral_image(hc_img)
  [area1, area2, area3] = get_haar_params(dark_w=2, bright_w=4, h=8) # площади каждой области
  square1 = area1[1][0] * (area1[1][1] - area1[0][1])
  square2 = area2[1][0] * (area2[1][1] - area2[0][1])
  square3 = area3[1][0] * (area3[1][1] - area3[0][1])
  # в цикле поворачиваем признак хаара на 10 градусов за итерацию
  # а потом с помощью признака пытаемся найти объекты на изображении 
  for angle in range(0, 360, 10):
    rotated_haar = transform.rotate(np.zeros(area3[1], dtype=np.uint8), angle, resize=True,
                                    preserve_range=False) # поворачиваем 
    window_w, window_h = rotated_haar.shape # получаем размеры поверну-того признака
          # с шагом 5 смотрим результат применения признака
    for x in range(0, hc_img.shape[0] - 2 * window_w - 1, 5):
      for y in range(0, hc_img.shape[1] - 2 * window_h - 1, 5): # Средний цвет пикселя первой светлой части
        mean_bright1 = transform.integrate(int_img, (x, y), (x + area1[1][0], y + area1[1][1])) / square1
# Средний цвет пикселя темной части
        mean_dark = transform.integrate(int_img, (x + area2[0][0], y + area2[0][1]), (x + area2[1][0], y + area2[1][1])) / square2 
# Средний цвет пикселя второй светлой части
        mean_bright2 = transform.integrate(int_img, (x + area3[0][0], y + area3[0][1]),
                                           (x + area3[1][0], y + area3[1][1])) / square3
        mean_bright = (mean_bright1 + mean_bright2) / 2 
        if mean_bright - mean_dark > w_threshold:
          res_coords.append((x, y, (window_w, window_h)))
  res_coords = list(set(res_coords)) 
  cleaned_coords = remove_close_hits(res_coords)
  return cleaned_coords # массив координат с результатами работы детекторы




In [ ]:
def remove_close_hits(res_coords):
  new_coords = sorted(res_coords, key=operator.itemgetter(0)) # сортируем по первой координате и находим близкие координаты и удяляем близкие из них i = 1
  n = len(new_coords) 
  while i < n:
    x = new_coords[i][0] 
    y = new_coords[i][1]
    x1 = new_coords[i - 1][0] 
    y1 = new_coords[i - 1][1] 
    difx = abs(x - x1)
    dify = abs(y - y1)
    if (difx + dify) <= 20:
      del new_coords[i] # удаляем n -= 1
    else:
      i += 1
  return new_coords


def save_result(img, coords, im_no):
  for x, y, window_shape in coords: # итерируемся по координатам и рисуем вокруг обьектов контуры
    coord = x, y
    if coord != (-1, -1):
      rr, cc = draw.rectangle_perimeter(coord, extent=window_shape,
                                        shape=img.shape) # рисуем кон-тур с нужными характеристиками
                                        # зеленый цвет обводки 
      img[rr, cc] = (0, 255, 0)
    imsave("cars_detected", im_no, img)


if name == ' main ': 
  main()


NameError: ignored